In [1]:
from sympy import Symbol, symbols, integrate, diff, simplify
from sympy.matrices import Matrix
from sympy.interactive.printing import init_printing

# init_printing(use_unicode=True, wrap_line=False)

In [2]:
# Координата по длине балочного элемента.
x = Symbol('x')

# Полиномы поля перемещений балочного элемента (без постоянных коэффициентов).
Q = Matrix([[1, x, 0, 0,    0,      0],  # Продольное перемещение (tx).
            [0, 0, 1, x, x**2,   x**3],  # Поперечное перемещение (ty).	
            [0, 0, 0, 1,  2*x, 3*x**2]]) # Поворот сечения (rz).
Q

Matrix([
[1, x, 0, 0,    0,      0],
[0, 0, 1, x, x**2,   x**3],
[0, 0, 0, 1,  2*x, 3*x**2]])

In [3]:
# Перемещения начала и конца балочного элемента.
Z = Matrix(symbols('tx1, ty1, rz1, tx2, ty2, rz2'))
# Длина балочного элемента.
L = Symbol('L')
# Полиномы для перемещения начала и конца балочного элемента.
C = Q.subs(x, 0).col_join( Q.subs(x, L) )
C

Matrix([
[1, 0, 0, 0,    0,      0],
[0, 0, 1, 0,    0,      0],
[0, 0, 0, 1,    0,      0],
[1, L, 0, 0,    0,      0],
[0, 0, 1, L, L**2,   L**3],
[0, 0, 0, 1,  2*L, 3*L**2]])

In [4]:
# Выражение поля перемещений через перемещение начала и конца балочного элемента.
B = Q * C.inv() * Z
B

Matrix([
[                                                                                                           tx1*(1 - x/L) + tx2*x/L],
[rz1*(x - 2*x**2/L + x**3/L**2) + rz2*(-x**2/L + x**3/L**2) + ty1*(1 - 3*x**2/L**2 + 2*x**3/L**3) + ty2*(3*x**2/L**2 - 2*x**3/L**3)],
[         rz1*(1 - 4*x/L + 3*x**2/L**2) + rz2*(-2*x/L + 3*x**2/L**2) + ty1*(-6*x/L**2 + 6*x**2/L**3) + ty2*(6*x/L**2 - 6*x**2/L**3)]])

In [5]:
# Погонная изгибная жесткость балки.
EF = Symbol('EI')
# Погонная осевая жесткость балки.
EI = Symbol('EF')
# Энергия деформации балки.
Energy  = EF/2 * integrate(diff(B[0], x)**2, (x, 0, L))
Energy += EI/2 * integrate(diff(B[1], x, 2)**2, (x, 0, L))
simplify(Energy)

(4*EF*(L**2*rz1**2 + L**2*rz1*rz2 + L**2*rz2**2 + 3*L*rz1*ty1 - 3*L*rz1*ty2 + 3*L*rz2*ty1 - 3*L*rz2*ty2 + 3*ty1**2 - 6*ty1*ty2 + 3*ty2**2) + EI*L**2*(tx1 - tx2)**2)/(2*L**3)

In [6]:
# Приращения энергии деформации балки по координатам.
dEnergy = Matrix( len(Z), 1, lambda i, j: diff(Energy, Z[i]) )
simplify(dEnergy)

Matrix([
[                           EI*(tx1 - tx2)/L],
[  6*EF*(L*rz1 + L*rz2 + 2*ty1 - 2*ty2)/L**3],
[2*EF*(2*L*rz1 + L*rz2 + 3*ty1 - 3*ty2)/L**2],
[                          EI*(-tx1 + tx2)/L],
[ 6*EF*(-L*rz1 - L*rz2 - 2*ty1 + 2*ty2)/L**3],
[2*EF*(L*rz1 + 2*L*rz2 + 3*ty1 - 3*ty2)/L**2]])

In [7]:
# Матрица жесткости элемента.
K = simplify( Matrix(len(Z), len(Z), lambda i, j: (dEnergy - dEnergy.subs(Z[i], 0))[j]) )
for z in Z:
	K = K.subs(z, 1)
K

Matrix([
[ EI/L,           0,          0, -EI/L,           0,          0],
[    0,  12*EF/L**3,  6*EF/L**2,     0, -12*EF/L**3,  6*EF/L**2],
[    0,   6*EF/L**2,     4*EF/L,     0,  -6*EF/L**2,     2*EF/L],
[-EI/L,           0,          0,  EI/L,           0,          0],
[    0, -12*EF/L**3, -6*EF/L**2,     0,  12*EF/L**3, -6*EF/L**2],
[    0,   6*EF/L**2,     2*EF/L,     0,  -6*EF/L**2,     4*EF/L]])